In [ ]:
# !pip install oracledb
# !pip install requests beautifulsoup4
# !pip install selenium

  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 9.4/9.4 MB 49.9 MB/s eta 0:00:00
Using cached trio-0.30.0-py3-none-any.whl (499 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cache


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 남성 의상 데이터 추출

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import oracledb

import re
from bs4 import BeautifulSoup

# DB 연결 설정

conn = oracledb.connect(
    user='coordi',
    password='12345',
    dsn='localhost/xe'
)
cursor = conn.cursor()

# INSERT SQL 남성
insert_sql = """
    INSERT INTO products_male(
        product_id,
        category_id,
        brand_name,
        product_name,
        image_url,
        link_url
    ) VALUES (
        :productId,
        :categoryId,
        :brandName,
        :productName,
        :imageUrl,
        :linkUrl
    )
"""

search_keywords = ["남성 민소매","남성 반소매","남성 헨리넥 반소매","남성 피케/카라티","남성 반팔 니트/스웨터","남성 반팔 셔츠","남성 반소매 가디건","남성 얇은 셔츠","남성 린넨 셔츠","남성 긴소매","남성 셔츠","남성 얇은 니트/스웨터","남성 얇은 가디건","남성 바람막이","남성 가디건","남성 니트/스웨터","남성 후드티","남성 맨투맨","남성 블레이저","남성 트렌치코트","남성 가죽자켓","남성 울코트","남성 패딩코트","남성 패딩","남성 면반바지","남성 데님 반바지","남성 린넨 팬츠","남성 화이트 데님","남성 얇은 데님","남성 얇은 슬랙스","남성 데님","남성 코튼 팬츠","남성 슬랙스","남성 스웨트팬츠","남성 두꺼운 데님","남성 울팬츠","남성 샌들/슬리퍼","남성 크록스","남성 스니커즈","남성 운동화","남성 구두","남성 부츠/워커","남성 패딩/퍼"]
local_id = 1

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 백그라운드 실행
driver = webdriver.Chrome(options=options)
driver.maximize_window()

for idx, keyword in enumerate(search_keywords, 1):
    driver.get('http://www.musinsa.com/')

    driver.find_element(By.CLASS_NAME, 'sc-dzagxs-2').click()
    search = driver.find_element(By.CLASS_NAME, 'sc-97s1c6-2')
    search.send_keys(keyword, Keys.ENTER)
    time.sleep(2)

    # 페이지를 두 번 스크롤하여 콘텐츠 로드
    def scroll(times=2, pause=2):
        for _ in range(times):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(pause)
    scroll(2, 2)

    # 데이터 수집
    soup = BeautifulSoup(driver.page_source,"html.parser")

    # 전체 상품 영역 가져오기
    products = soup.find_all("div", class_="sc-d36st-1")

    product_all = []

    for product in products:
        img = product.select_one("img")
        # 상품경로
        detail = product.select_one("a") 
        link_url = detail.get("href")   
        # product_id = detail.get("data-item-id")   
        product_id = local_id
        local_id += 1
        # 브랜드명
        brand_name = product.select_one("a > span").text.strip()
        # 상품명
        product_name = product.select_one('div.sc-cMuefe.jIlLeV > a.gtm-select-item > span').text.strip()
        # 이미지경로
        image_url = img.get('src').split("?")[0]
        # print(product_id,image_url,product_name,link_url,brand_name)

        product_all.append([product_id,idx,brand_name,product_name,image_url,link_url])

    time.sleep(3)

    # DB 코드
    cursor.executemany("INSERT INTO products_male(product_id,category_id,brand_name,product_name,image_url,link_url) VALUES (:1, :2, :3, :4, :5, :6)",product_all)
    conn.commit()


### 여성 의상 데이터 추출

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import oracledb

import re
from bs4 import BeautifulSoup

# DB 연결 설정
conn = oracledb.connect(
    user='coordi',
    password='12345',
    dsn='localhost/xe'
)
cursor = conn.cursor()

# INSERT SQL 여성
insert_sql = """
    INSERT INTO products_female(
        product_id,
        category_id,
        brand_name,
        product_name,
        image_url,
        link_url
    ) VALUES (
        :productId,
        :categoryId,
        :brandName,
        :productName,
        :imageUrl,
        :linkUrl
    )
"""

search_keywords = ["여성 나시","여성 반소매","여성 헨리넥 반소매","여성 반소매 블라우스","여성 피케/카라 원피스","여성 여름 가디건","여성 원피스","여성 맥시드레스","여성 반팔 니트/스웨터","여성 긴소매","여성 블라우스","여성 얇은 니트/스웨터","여성 얇은 가디건","여성 바람막이","여성 가디건","여성 니트/스웨터","여성 후드티","여성 블레이저","여성 트렌치코트","여성 가죽자켓","여성 울코트","여성 패딩","여성 반바지","여성 숏스커트","여성 린넨 팬츠","여성 화이트 데님","여성 얇은 데님","여성 얇은 슬랙스","여성 롱스커트","여성 데님","여성 코튼 팬츠","여성 슬랙스","여성 스웨트팬츠","여성 두꺼운 데님","여성 울팬츠","여성 샌들/슬리퍼","여성 크록스","여성 스니커즈","여성 오픈토힐","여성 운동화","여성 구두","여성 앵클부츠","여성 부츠","여성 패딩/퍼"]
local_id = 1

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 백그라운드 실행
driver = webdriver.Chrome(options=options)
driver.maximize_window()

for idx, keyword in enumerate(search_keywords, 1):
    driver.get('http://www.musinsa.com/')

    driver.find_element(By.CLASS_NAME, 'sc-dzagxs-2').click()
    search = driver.find_element(By.CLASS_NAME, 'sc-97s1c6-2')
    search.send_keys(keyword, Keys.ENTER)
    time.sleep(2)

    # 페이지를 두 번 스크롤하여 콘텐츠 로드
    def scroll(times=2, pause=2):
        for _ in range(times):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(pause)
    scroll(2, 2)

    # 데이터 수집
    soup = BeautifulSoup(driver.page_source,"html.parser")

    # 전체 상품 영역 가져오기
    products = soup.find_all("div", class_="sc-d36st-1")

    product_all = []

    for product in products:
        img = product.select_one("img")
        # 상품경로
        detail = product.select_one("a") 
        link_url = detail.get("href")   
        # product_id = detail.get("data-item-id")   
        product_id = local_id
        local_id += 1
        # 브랜드명
        brand_name = product.select_one("a > span").text.strip()
        # 상품명
        product_name = product.select_one('div.sc-cMuefe.jIlLeV > a.gtm-select-item > span').text.strip()
        # 이미지경로
        image_url = img.get('src').split("?")[0]
        # print(product_id,image_url,product_name,link_url,brand_name)

        product_all.append([product_id,idx,brand_name,product_name,image_url,link_url])

    time.sleep(3)

    # DB 코드
    cursor.executemany("INSERT INTO products_female(product_id,category_id,brand_name,product_name,image_url,link_url) VALUES (:1, :2, :3, :4, :5, :6)",product_all)
    conn.commit()